In [1]:
import json

import pandas as pd
import numpy as np
import networkx as nx

import torch
from torch_geometric.nn import GATConv, GATv2Conv
from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import GraphSAINTRandomWalkSampler, GraphSAINTNodeSampler
from matplotlib import pyplot as plt
import numpy as np
from train_utils import *
from product_graph import *
from torch_geometric.data import HeteroData
from tqdm import tqdm
from torch_geometric.utils import to_dense_adj
from torch.nn import MSELoss
from product_graph import generate_parametric_product_graph
import networkx as nx
from torch_geometric.utils import from_networkx

In [2]:
class GATv3Conv(torch.nn.Module):
    def __init__(self, in_channels, out_channels,concat = True, heads=1, dropout = 0.5) -> None:
        super().__init__()
        self.beta = torch.nn.Parameter(torch.tensor(0.5))
        self.conv = GATv2Conv(in_channels, out_channels, heads, concat,add_self_loops=False)

    def forward(self, x, edge_index, edge_weights):
        H, C = self.conv.heads, self.conv.out_channels

        if isinstance(x, torch.Tensor):
            assert x.dim() == 2
            x_l = self.conv.lin_l(x).view(-1, H, C)
            if self.conv.share_weights:
                x_r = x_l
            else:
                x_r = self.conv.lin_r(x).view(-1, H, C)
        else:
            raise TypeError("x must be a Tensor")

        assert x_l is not None
        assert x_r is not None

        # edge_updater_type: (x: PairTensor, edge_attr: OptTensor)
        alpha = self.conv.edge_updater(edge_index, x=(x_l, x_r), edge_attr=None)

        alpha = (1-self.beta) * alpha + self.beta * edge_weights

        # propagate_type: (x: PairTensor, alpha: Tensor)
        out = self.conv.propagate(edge_index, x=(x_l, x_r), alpha=alpha)

        if self.conv.concat:
            out = out.view(-1, self.conv.heads * self.conv.out_channels)
        else:
            out = out.mean(dim=1)

        if self.conv.bias is not None:
            out = out + self.conv.bias

        return out
    

class GATNN(torch.nn.Module):
    def __init__(self, hidden_size, in_dim, out_dim,in_head = 8,out_head = 1) -> None:
        super().__init__()
        self.hid = hidden_size
        self.in_head = in_head
        self.out_head = out_head
        
        
        self.conv1 = GATv3Conv(in_dim, self.hid, heads=self.in_head, dropout=0.6)
        self.conv2 = GATv3Conv(self.hid*self.in_head, out_dim, concat=False,
                             heads=self.out_head, dropout=0.6)

    def forward(self, x, edge_index, edge_weight):
        x = self.conv1(x, edge_index, edge_weight)
        x = self.conv2(x, edge_index, edge_weight)
        return x

In [3]:
import pandas as pd
dynamic_data = torch.tensor(np.load("data/preprocessed/dynamic_data.npy", allow_pickle=True))
S = torch.tensor(np.load("data/adjacency/coords_features.npy", allow_pickle=False))


calendar_df = pd.read_csv('data/preprocessed/calendar.csv')
data = Data(x = dynamic_data)
edge_index = torch.nonzero(torch.tensor(S), as_tuple=False).t().contiguous()
edge_weight = S[edge_index[0], edge_index[1]]
data.edge_index = edge_index
data.edge_weight = edge_weight

/var/folders/k7/vcjbn_bd7wdb2g9h92g_pkj80000gn/T/ipykernel_78384/1233144186.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index = torch.nonzero(torch.tensor(S), as_tuple=False).t().contiguous()


In [4]:
print(dynamic_data.shape)

torch.Size([2628, 279])


In [5]:
criterion = MSELoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GATNN(hidden_size=8, in_dim = 4, out_dim = dynamic_data.shape[1]).to(device)

loader = GraphSAINTNodeSampler (
    data,
    batch_size=200,
    num_steps=6,
)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

temporal_adj = np.array([[0, 0, 0, 0],
                [1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 1, 0]])



In [19]:
product_graph = generate_parametric_product_graph(s00 = 0, s01 = 1, s10 = 1, s11 = 1, A_T = temporal_adj, A_N = S.numpy(), spatial_graph = nx.from_numpy_array(S.numpy()))
product_data = from_networkx(product_graph)

KeyboardInterrupt: 

In [ ]:
train_loss = []
for epoch in range(20):
    model.train()
    total_loss = 0
    for i, batch in enumerate(loader):
        batch_loss = 0
        dataset = create_forecasting_dataset(batch.x.T,
                                        splits = None,
                                        pred_horizen= 1,
                                        obs_window= 4,
                                        verbose = 0)
        batch_sample = Data(x = torch.tensor(dataset['trn']['data']), y = torch.tensor(dataset['trn']['labels']), 
                            edge_index= batch.edge_index, edge_weight = batch.edge_weight)

        batch_adj = to_dense_adj(batch_sample.edge_index, edge_attr=batch_sample.edge_weight).squeeze(dim = 0)

        batch_adj = batch_adj.numpy()
        
        product_graph = generate_parametric_product_graph(s00 = 0, s01 = 1, s10 = 1, s11 = 1, A_T = temporal_adj, A_N = batch_adj, spatial_graph = nx.from_numpy_array(batch_adj))
        product_data = from_networkx(product_graph)
        for i in range(batch_sample.x.shape[0]):
        
            optimizer.zero_grad()
            out = model(batch_sample.x[i].float(), product_data.edge_index,product_data.edge_weight)
            loss = criterion(out, batch_sample.y[i].float())
            batch_loss += loss
            
            if epoch%10 == 0:
                print(f'Epoch: {epoch} Loss: {loss}')
        
            loss.backward()
            optimizer.step()
            
        total_loss += batch_loss
    train_loss.append(total_loss)

AttributeError: 'Graph' object has no attribute 'shape'

In [ ]:
model.eval()
_, pred = model(x_test).max(dim=1)
correct = float(pred.eq(y_test).sum().item())
acc = correct / x_test.sum().item()
print('Accuracy: {:.4f}'.format(acc))

In [201]:
x = torch.tensor([[100], 
                  [1],
                  [2],
                  [3]]).float()

edge_index = torch.tensor([[0, 1, 1, 2, 3, 2], 
                           [1, 0, 2, 1, 2, 3]])

edge_weights = torch.tensor([[0.3],
                             [0.3],
                             [0.9],
                             [0.9],
                             [0.1],
                             [0.1]])

gat_net(x, edge_index, edge_weights)

tensor([[-5.4292],
        [-0.9369],
        [-4.6689],
        [-0.5501]], grad_fn=<AddBackward0>)